In [1]:
import numpy as np
from numba import cfunc
from cminpack_numba import hybrd_sig, hybrd1, hybrd
from scipy.optimize import root

def func(x):
    return [x[0] * np.cos(x[1]) - 4, x[1] * x[0] - x[1] - 5]


@cfunc(hybrd_sig)
def func_numba(udata, n, x, fvec, iflag):
    fvec[0] = x[0] * np.cos(x[1]) - 4
    fvec[1] = x[1] * x[0] - x[1] - 5
    return 0

In [2]:
root(func, [1, 1], method='hybr')

 message: The solution converged.
 success: True
  status: 1
     fun: [ 3.732e-12  1.617e-11]
       x: [ 6.504e+00  9.084e-01]
    nfev: 17
    fjac: [[-5.625e-01 -8.268e-01]
           [ 8.268e-01 -5.625e-01]]
       r: [-1.091e+00 -1.762e+00 -7.374e+00]
     qtf: [ 6.257e-08  2.401e-08]

In [3]:
%timeit root(func, [1, 1], method='hybr')

26.5 µs ± 654 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [4]:
print(hybrd1(func_numba.address, np.array([1.0, 1.0])))
%timeit hybrd1(func_numba.address, np.array([1.0, 1.0]))

(array([6.50409711, 0.90841421]), array([-7.39657224e-11,  6.40909548e-12]), 1)
2.9 µs ± 17.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
print(hybrd(func_numba.address, np.array([1.0, 1.0])))
%timeit hybrd(func_numba.address, np.array([1.0, 1.0]))

(array([6.50409711, 0.90841421]), array([3.73212572e-12, 1.61701763e-11]), 1, array([[-0.56248005, -0.82681085],
       [ 0.82681085, -0.56248005]]), array([-1.0907073 , -1.7621827 , -7.37420598]), array([6.25677405e-08, 2.40104780e-08]), 17)
31.3 µs ± 97 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
from llvmlite import binding
from ctypes.util import find_library
from numba import types, njit
from cminpack_numba.utils import ptr_from_val, val_from_ptr

_cminpack_prefix = {types.float32: "s", types.float64: ""}


def _cminpack_func(func, dtype):
    return f"{_cminpack_prefix[dtype]}{func}"

binding.load_library_permanently(find_library('cminpack'))
def sig_hybrd(dtype: types.Float) -> types.ExternalFunction:  # pylint: disable= C0116
    sig = types.int32(
        types.long_,  # fcn
        types.voidptr,  # *p / *udata
        types.int32,  # n
        types.CPointer(dtype),  # *x
        types.CPointer(dtype),  # *fvec
        dtype,  # xtol
        types.int32,  # maxfev
        types.int32,  # ml
        types.int32,  # mu
        dtype,  # epsfcn
        types.CPointer(dtype),  # *diag
        types.int32,  # mode
        dtype,  # factor
        types.int32,  # nprint
        types.CPointer(types.int32),  # *nfev
        types.CPointer(dtype),  # *fjac
        types.int32,  # ldfjac
        types.CPointer(dtype),  # *r
        types.int32,  # lr
        types.CPointer(dtype),  # *qtf
        types.CPointer(dtype),  # *wa1
        types.CPointer(dtype),  # *wa2
        types.CPointer(dtype),  # *wa3
        types.CPointer(dtype),  # *wa4
    )
    return types.ExternalFunction(_cminpack_func("hybrd", dtype), sig)

_hybrd_cfunc = sig_hybrd(types.float64)

@njit
def _hybrd_args(x):
    n = np.int32(x.size)
    fvec = np.empty(n, dtype=x.dtype)
    nfevptr = ptr_from_val(np.int32(0))
    fjac = np.empty((n, n), dtype=x.dtype)
    ldfjac = n
    lr = (n*(n+1))//2
    r = np.empty(lr, dtype=x.dtype)
    qtf = np.empty(n, dtype=x.dtype)
    wa = np.empty(4*n, dtype=x.dtype)
    wa1 = wa[:n]
    wa2 = wa[n:2*n]
    wa3 = wa[2*n:3*n]
    wa4 = wa[3*n:]

    return n, fvec, nfevptr, fjac, ldfjac, lr, r, qtf, wa1, wa2, wa3, wa4

@njit
def _hybrd(fcn, x, xtol=1.49012e-8, maxfev=-1, ml=None, mu=None, epsfcn=None, diag=None,
          mode=1, factor=100., nprint=0, udata=None):
    n, fvec, nfevptr, fjac, ldfjac, lr, r, qtf, wa1, wa2, wa3, wa4 = _hybrd_args(
        x)
    if diag is None:
        diag = np.ones(n, dtype=x.dtype)
    if epsfcn is None:
        epsfcn = np.finfo(x.dtype).eps
    # if not maxfev:
    #     maxfev = 200 * (n + 1)
    if ml is None:
        ml = n
    if mu is None:
        mu = n
    if udata is None:
        udata = np.empty(1, dtype=np.int64)
    x0 = x.copy()
    info = _hybrd_cfunc(fcn, udata.ctypes, n, x0.ctypes, fvec.ctypes, xtol, maxfev, ml, mu, epsfcn, diag.ctypes, mode, factor,
                  nprint, nfevptr, fjac.ctypes, ldfjac, r.ctypes, lr, qtf.ctypes, wa1.ctypes, wa2.ctypes, wa3.ctypes, wa4.ctypes)
    nfev = val_from_ptr(nfevptr)
    return x0, fvec, info, fjac, r, qtf, nfev
    

print(_hybrd(func_numba.address, np.array([1.0, 1.0])))
%timeit _hybrd(func_numba.address, np.array([1.0, 1.0]))

(array([1., 1.]), array([0., 0.]), 0, array([[0., 0.],
       [0., 0.]]), array([0., 0., 0.]), array([0., 0.]), 0)
29.3 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
